In [22]:
import pandas as pd
from tensorflow.keras.utils import get_file

pd.set_option('display.max_columns', 6)
pd.set_option('display.max_rows', 5)

try:
    path = get_file('kdd-with-columns.csv', origin=\
    'https://github.com/jeffheaton/jheaton-ds2/raw/main/'\
    'kdd-with-columns.csv',archive_format=None)
except:
    print('Error downloading')
    raise
    
print(path) 

df = pd.read_csv(path)

print("Read {} rows.".format(len(df)))
df.dropna(inplace=True,axis=1) 

pd.set_option('display.max_columns', 5)
pd.set_option('display.max_rows', 5)
df

/root/.keras/datasets/kdd-with-columns.csv
Read 494021 rows.


,duration,protocol_type,...,dst_host_srv_rerror_rate,outcome
0,0,tcp,...,0.0,normal.
1,0,tcp,...,0.0,normal.
...,...,...,...,...,...
494019,0,tcp,...,0.0,normal.
494020,0,tcp,...,0.0,normal.


In [23]:
df.groupby('outcome')['outcome'].count()

outcome
back.               2203
buffer_overflow.      30
                    ... 
warezclient.        1020
warezmaster.          20
Name: outcome, Length: 23, dtype: int64

In [24]:
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd
    
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = f"{name}-{x}"
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)


In [25]:
pd.set_option('display.max_columns', 6)
pd.set_option('display.max_rows', 5)

for name in df.columns:
  if name == 'outcome':
    pass
  elif name in ['protocol_type','service','flag','land','logged_in',
                'is_host_login','is_guest_login']:
    encode_text_dummy(df,name)
  else:
    encode_numeric_zscore(df,name)    

df.dropna(inplace=True,axis=1)
df[0:5]

,duration,src_bytes,dst_bytes,...,is_host_login-0,is_guest_login-0,is_guest_login-1
0,-0.067792,-0.002879,0.138664,...,1,1,0
1,-0.067792,-0.002820,-0.011578,...,1,1,0
2,-0.067792,-0.002824,0.014179,...,1,1,0
3,-0.067792,-0.002840,0.014179,...,1,1,0
4,-0.067792,-0.002842,0.035214,...,1,1,0


In [26]:
normal_mask = df['outcome']=='normal.'
attack_mask = df['outcome']!='normal.'

df.drop('outcome',axis=1,inplace=True)

df_normal = df[normal_mask]
df_attack = df[attack_mask]

print(f"Normal count: {len(df_normal)}")
print(f"Attack count: {len(df_attack)}")

Normal count: 97278
Attack count: 396743


In [27]:
x_normal = df_normal.values
x_attack = df_attack.values

In [28]:
from sklearn.model_selection import train_test_split

x_normal_train, x_normal_test = train_test_split(
    x_normal, test_size=0.25, random_state=42)

In [29]:
print(f"Normal train count: {len(x_normal_train)}")
print(f"Normal test count: {len(x_normal_test)}")

Normal train count: 72958
Normal test count: 24320


In [30]:
from sklearn import metrics
import numpy as np
import pandas as pd
from IPython.display import display, HTML 
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation

model = Sequential()
model.add(Dense(25, input_dim=x_normal.shape[1], activation='relu'))
model.add(Dense(3, activation='relu')) 
model.add(Dense(25, activation='relu'))
model.add(Dense(x_normal.shape[1])) 
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(x_normal_train,x_normal_train,verbose=1,epochs=100)

Epoch 1/100
2280/2280 [==============================] - 5s 2ms/step - loss: 0.3603
Epoch 2/100
2280/2280 [==============================] - 4s 2ms/step - loss: 0.2705
Epoch 3/100
2280/2280 [==============================] - 4s 2ms/step - loss: 0.2306
Epoch 4/100
2280/2280 [==============================] - 4s 2ms/step - loss: 0.2114
Epoch 5/100
2280/2280 [==============================] - 4s 2ms/step - loss: 0.1930
Epoch 6/100
2280/2280 [==============================] - 4s 2ms/step - loss: 0.1915
Epoch 7/100
2280/2280 [==============================] - 4s 2ms/step - loss: 0.1907
Epoch 8/100
2280/2280 [==============================] - 4s 2ms/step - loss: 0.1811
Epoch 9/100
2280/2280 [==============================] - 4s 2ms/step - loss: 0.1825
Epoch 10/100
2280/2280 [==============================] - 4s 2ms/step - loss: 0.1777
Epoch 11/100
2280/2280 [==============================] - 4s 2ms/step - loss: 0.1758
Epoch 12/100
2280/2280 [==============================] - 4s 2ms/step - lo

In [31]:
pred = model.predict(x_normal_test)
score1 = np.sqrt(metrics.mean_squared_error(pred,x_normal_test))
pred = model.predict(x_normal)
score2 = np.sqrt(metrics.mean_squared_error(pred,x_normal))
pred = model.predict(x_attack)
score3 = np.sqrt(metrics.mean_squared_error(pred,x_attack))
print(f"Out of Sample Normal Score (RMSE): {score1}")
print(f"Insample Normal Score (RMSE): {score2}")
print(f"Attack Underway Score (RMSE): {score3}")

Out of Sample Normal Score (RMSE): 0.33033253286750647
Insample Normal Score (RMSE): 0.3384370285671334
Attack Underway Score (RMSE): 0.5721651382754654
